<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Librerías" data-toc-modified-id="Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Librerías</a></span></li><li><span><a href="#Cargar-datos" data-toc-modified-id="Cargar-datos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Cargar datos</a></span></li><li><span><a href="#Two-asset-portfolio-MSR" data-toc-modified-id="Two-asset-portfolio-MSR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Two-asset portfolio MSR</a></span><ul class="toc-item"><li><span><a href="#Retornos-reales" data-toc-modified-id="Retornos-reales-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Retornos reales</a></span></li><li><span><a href="#Retornos-estimados" data-toc-modified-id="Retornos-estimados-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Retornos estimados</a></span></li><li><span><a href="#MSR" data-toc-modified-id="MSR-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>MSR</a></span></li></ul></li><li><span><a href="#EW" data-toc-modified-id="EW-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>EW</a></span></li><li><span><a href="#GMV" data-toc-modified-id="GMV-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>GMV</a></span></li></ul></div>

# Global Minimum Volatility Portfolio

Falta de robustez del procedimiento de Markowitz y de la cartera GMV
El procedimiento de Markowitz en la práctica tiende a fallar porque es dificil conocer los rendimientos esperados y la covarianza esperada de antemano (las estimaciones contienen casi con toda seguridad algún error de estimación y este  procedimiento es muy sensible a estos errores).

## Librerías

In [1]:
import pandas as pd
from modules import edhec_risk_kit as erk
import plotly 
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import *

# Modo desarrollo del módulo edhec_risk_kit
%matplotlib inline
%load_ext autoreload
%autoreload 2
plotly.offline.init_notebook_mode(connected = True)

## Cargar datos


In [2]:
df_ind = erk.get_ind_returns(filename='../data/ind30_m_vw_rets.csv')
df_er = erk.annualize_rets(df_ind['1996':'2000'], 12)
df_cov = df_ind['1996':'2000'].cov()

## Two-asset portfolio MSR

El caso más sencillo es trabajar con una cartera de dos activos para tratar de calcular los pésos óptimos si se conocieses de forma exacta los retonos esperados.

### Retornos reales


En este caso se va a trabajar con los retornos reales que se obtuvieron durante un período de tiempo específico, es decir, no tienen error de estimación

In [3]:
l = ['Food', 'Steel']

# Obtener pesos de la cartera MSR.
w_msr = erk.msr(risk_free_rate=0.1, expected_returns=df_er[l], covmat=df_cov.loc[l][l])
ret_msr = erk.portfolio_return(w_msr, df_er[l])
vol_msr = erk.portfolio_volatility(w_msr, df_cov.loc[l][l])

# Resumen
erk.resume_ef(expected_returns=df_er[l], weights=w_msr, ret=ret_msr, vol=vol_msr, rentabilidad_individual=True)

Assets: ['Food', 'Steel']


Optimal Weights:
	Food = 75.04%
	Steel = 24.96%


Rentabilidad = 11.66%
Volatilidad = 4.41%


Rentabilidad individual:
	Food = 11.68%
	Steel = 11.58%


### Retornos estimados

En esta ocasión, se va a trabajar con una "estimación de retornos" para comprobar el efecto que tiene sobre el método MSR.

Se modifica ligeramente el valor del retorno para simular una estimación en lugar del valor real
y comprobar la variación en el resultado.

In [4]:
l = ['Food', 'Steel']

# Modificamos ligeramente los returnos esperados.
er = df_er[l].copy()
er['Food'] = 0.11
er['Steel'] = 0.12

# Obtener pesos de la cartera MSR.
w_msr = erk.msr(risk_free_rate=0.1, expected_returns=er, covmat=df_cov.loc[l][l])
ret_msr = erk.portfolio_return(w_msr, df_er[l])
vol_msr = erk.portfolio_volatility(w_msr, df_cov.loc[l][l])


erk.resume_ef(expected_returns=er, weights=w_msr, ret=ret_msr, vol=vol_msr, rentabilidad_individual=True)

Assets: ['Food', 'Steel']


Optimal Weights:
	Food = 57.93%
	Steel = 42.07%


Rentabilidad = 11.64%
Volatilidad = 4.68%


Rentabilidad individual:
	Steel = 12.00%
	Food = 11.00%


__Por lo tanto, un pequeño cambio en los rendimientos esperado produce un cambio bastante dramático en el valor de los pesos__

In [5]:
l = ['Food', 'Steel']

# Modificamos ligeramente los returnos esperados.
er = df_er[l].copy()
er['Food'] = 0.10
er['Steel'] = 0.13

# Obtener pesos de la cartera MSR.
w_msr = erk.msr(risk_free_rate=0.1, expected_returns=er, covmat=df_cov.loc[l][l])
ret_msr = erk.portfolio_return(w_msr, df_er[l])
vol_msr = erk.portfolio_volatility(w_msr, df_cov.loc[l][l])

erk.resume_ef(expected_returns=er, weights=w_msr, ret=ret_msr, vol=vol_msr, rentabilidad_individual=False)

Assets: ['Food', 'Steel']


Optimal Weights:
	Steel = 100.00%


Rentabilidad = 11.58%
Volatilidad = 8.50%


In [6]:
l = ['Food', 'Steel']

# Modificamos ligeramente los returnos esperados.
er = df_er[l].copy()
er['Food'] = 0.13
er['Steel'] = 0.10

# Obtener pesos de la cartera MSR.
w_msr = erk.msr(risk_free_rate=0.1, expected_returns=er, covmat=df_cov.loc[l][l])
ret_msr = erk.portfolio_return(w_msr, df_er[l])
vol_msr = erk.portfolio_volatility(w_msr, df_cov.loc[l][l])

erk.resume_ef(expected_returns=er, weights=w_msr, ret=ret_msr, vol=vol_msr, rentabilidad_individual=False)

Assets: ['Food', 'Steel']


Optimal Weights:
	Food = 100.00%


Rentabilidad = 11.68%
Volatilidad = 5.11%


La gran influencia del valor de los retornos estimados en el valor de los pesos óptimos para obtener la cartera MSR es conocido como __Error maximizando la naturaleza de Markowitz__, es decir, un error de estimación pequeño produce cambios muy significativos en como se lleva a cabo la distribución de la cartera.

### MSR 

In [7]:
# Obtener pesos de la cartera MSR.
w_msr = erk.msr(risk_free_rate=0.1, expected_returns=df_er, covmat=df_cov)
ret_msr = erk.portfolio_return(w_msr, df_er)
vol_msr = erk.portfolio_volatility(w_msr, df_cov)


erk.resume_ef(expected_returns=df_er, weights=w_msr, ret=ret_msr, vol=vol_msr, rentabilidad_individual=False)

Assets: ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']


Optimal Weights:
	Hlth = 45.60%
	ElcEq = 23.38%
	Coal = 13.87%
	Util = 8.94%
	Rtail = 6.66%
	BusEq = 1.55%


Rentabilidad = 26.47%
Volatilidad = 4.57%


In [8]:
# Visualización
erk.plot_ef(n_points = 20, 
            expected_returns = df_er, 
            covmat = df_cov, 
            title='Efficient Frontier Portfolio Maximum Sharpe Ratio', 
            show_cml=True, 
            risk_free_rate=0.1)

## EW

Cartera que distribuye el mismo peso entre todos los activos que lo consituyen. Es un método que permite evitar la estimación de los retornos esperados

In [9]:
# Visualización
erk.plot_ef(n_points = 20, 
            expected_returns = df_er, 
            covmat = df_cov, 
            title='Efficient Frontier Portfolio Maximum Sharpe Ratio', 
            show_cml=True, 
            risk_free_rate=0.1,
            show_ew=True)

## GMV

In [10]:
# Visualización
erk.plot_ef(n_points = 20, 
            expected_returns = df_er, 
            covmat = df_cov, 
            title='Efficient Frontier Portfolio Maximum Sharpe Ratio', 
            show_cml=True, 
            risk_free_rate=0.1,
            show_ew=True,
            show_gmv=True)

In [11]:
# Obtener pesos de la cartera MSR.
w_msr = erk.msr(risk_free_rate=0.1, expected_returns=df_er, covmat=df_cov)
ret_msr = erk.portfolio_return(w_msr, df_er)
vol_msr = erk.portfolio_volatility(w_msr, df_cov)



erk.resume_ef(expected_returns=df_er, weights=w_msr, ret=ret_msr, vol=vol_msr, rentabilidad_individual=False)

Assets: ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']


Optimal Weights:
	Hlth = 45.60%
	ElcEq = 23.38%
	Coal = 13.87%
	Util = 8.94%
	Rtail = 6.66%
	BusEq = 1.55%


Rentabilidad = 26.47%
Volatilidad = 4.57%


In [12]:
w_gmv = erk.gmv(covmat=df_cov)
ret_gmv = erk.portfolio_return(w_gmv, df_er)
vol_gmv = erk.portfolio_volatility(w_gmv, df_cov)

erk.resume_ef(expected_returns=df_er, weights=w_gmv, ret=ret_gmv, vol=vol_gmv, rentabilidad_individual=False)

Assets: ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']


Optimal Weights:
	Util = 48.29%
	Meals = 11.52%
	BusEq = 10.13%
	Rtail = 8.39%
	Hshld = 7.82%
	Hlth = 6.49%
	Telcm = 3.15%
	Food = 3.13%
	Smoke = 0.84%
	Games = 0.24%


Rentabilidad = 15.83%
Volatilidad = 3.14%
